# Modelling

### Pipeline
- load the data
- replace null values
- separate categorical and numerical
- remove columns with very high entropy in categorical and numerical
- convert target to binary
- run pca on numerical
- one hot encoding on categorical
- train test separation
- Grid search for dense_nn configuration

### Imports

In [1]:
%load_ext autoreload

%autoreload 2

import os,sys,inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0,parentdir) 

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.metrics import precision_recall_fscore_support
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

import pickle

from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Lambda
from keras import regularizers
from keras.losses import mean_absolute_error

from keras.wrappers.scikit_learn import KerasClassifier


import math
import numpy as np

from etl.redundant_columns_remover import RedundantColumnsRemover



Using TensorFlow backend.
/home/paul/anaconda3/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:493: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/paul/anaconda3/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:494: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/paul/anaconda3/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:495: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/paul/anaconda3/lib/python3.6/site-packages/tensorflow/py

In [2]:
from etl.combined_transformer import CombinedTransformer

### Load Data

In [3]:
train_data = pd.read_csv("../data/loan-default-prediction/train_v2.csv")

/home/paul/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (135,204,274,417) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
train_id = train_data["id"]
train_loss = train_data["loss"]

train_data.drop(columns=["id", "loss"], inplace=True)

In [5]:
train_data, test_data, train_loss, test_loss = train_test_split(train_data, train_loss, test_size=0.33, random_state=42)

In [17]:
etl_pipeline = Pipeline([
    ("redundant_columns_remover", RedundantColumnsRemover()),
    ("combined_transformer", CombinedTransformer(["f776", "f777", "f725"])),
    ("pca", PCA(svd_solver='full')),
    ("standard_scaler", StandardScaler())
])
     

In [18]:
X_train = etl_pipeline.fit_transform(train_data)

/home/paul/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/data.py:2863: RuntimeWarning: divide by zero encountered in log
  loglike = -n_samples / 2 * np.log(x_trans.var())
/home/paul/anaconda3/lib/python3.6/site-packages/numpy/core/_methods.py:195: RuntimeWarning: overflow encountered in multiply
  x = um.multiply(x, x, out=x)


In [19]:
X_test = etl_pipeline.transform(test_data)

In [20]:
y_train_class = train_loss.astype("bool").astype("int")

In [21]:
y_test_class = test_loss.astype("bool").astype("int")

## Classification

### Logistic Regression Classifier

In [22]:
logistic_regression = LogisticRegression()
logistic_regression.fit(X_train, y_train_class)
precision_recall_fscore_support(y_test_class.values, logistic_regression.predict(X_test))

/home/paul/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


(array([0.92922034, 0.21692424]),
 array([0.8683752 , 0.35535659]),
 array([0.89776803, 0.26939731]),
 array([31567,  3239]))

### NN Classifier

Do the correlation between target and source columns

In [65]:
def get_classification_model():
    model=Sequential()

    model.add(Dense(256, input_dim=X_train.shape[1], kernel_regularizer=regularizers.l1_l2(l1=0.005, l2=0.001)))
    model.add(Activation("tanh"))
    model.add(Dense(128, kernel_regularizer=regularizers.l1_l2(l1=0.005, l2=0.001)))
    model.add(Activation("tanh"))
    model.add(Dense(1))
    model.add(Activation("sigmoid"))



    model.compile(optimizer='adam',
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    return model
classification_model = get_classification_model()

In [66]:
classification_model.fit(X_train, y_train_class, epochs=1000, batch_size=4096, validation_data=(X_test, y_test_class), class_weight={0:1., 1: 4.})

Train on 70665 samples, validate on 34806 samples
Epoch 1/1000
70665/70665 [==============================] - 4s 63us/step - loss: 38.7012 - acc: 0.5790 - val_loss: 29.7814 - val_acc: 0.7108
Epoch 2/1000
70665/70665 [==============================] - 4s 50us/step - loss: 24.0127 - acc: 0.8191 - val_loss: 17.1599 - val_acc: 0.8892
Epoch 3/1000
70665/70665 [==============================] - 4s 51us/step - loss: 13.1666 - acc: 0.9032 - val_loss: 8.3734 - val_acc: 0.9069
Epoch 4/1000
70665/70665 [==============================] - 4s 53us/step - loss: 6.1416 - acc: 0.9077 - val_loss: 3.4619 - val_acc: 0.9073
Epoch 5/1000
70665/70665 [==============================] - 4s 59us/step - loss: 2.8727 - acc: 0.9078 - val_loss: 1.7677 - val_acc: 0.9070
Epoch 6/1000
70665/70665 [==============================] - 4s 55us/step - loss: 1.6470 - acc: 0.9078 - val_loss: 0.9516 - val_acc: 0.9070
Epoch 7/1000
70665/70665 [==============================] - 4s 51us/step - loss: 1.0955 - acc: 0.9077 - val_los

70665/70665 [==============================] - 4s 52us/step - loss: 0.6686 - acc: 0.8997 - val_loss: 0.5024 - val_acc: 0.8940
Epoch 60/1000
70665/70665 [==============================] - 4s 51us/step - loss: 0.6685 - acc: 0.8993 - val_loss: 0.5050 - val_acc: 0.8945
Epoch 61/1000
70665/70665 [==============================] - 4s 52us/step - loss: 0.6681 - acc: 0.8971 - val_loss: 0.4881 - val_acc: 0.9015
Epoch 62/1000
70665/70665 [==============================] - 4s 52us/step - loss: 0.6664 - acc: 0.9007 - val_loss: 0.5020 - val_acc: 0.8952
Epoch 63/1000
70665/70665 [==============================] - 4s 50us/step - loss: 0.6654 - acc: 0.9009 - val_loss: 0.4998 - val_acc: 0.8963
Epoch 64/1000
70665/70665 [==============================] - 4s 53us/step - loss: 0.6643 - acc: 0.9003 - val_loss: 0.5025 - val_acc: 0.8958
Epoch 65/1000
70665/70665 [==============================] - 4s 55us/step - loss: 0.6628 - acc: 0.9000 - val_loss: 0.4936 - val_acc: 0.8993
Epoch 66/1000
70665/70665 [=======

70665/70665 [==============================] - 4s 50us/step - loss: 0.6392 - acc: 0.9038 - val_loss: 0.4858 - val_acc: 0.8997
Epoch 118/1000
70665/70665 [==============================] - 4s 50us/step - loss: 0.6378 - acc: 0.9036 - val_loss: 0.4800 - val_acc: 0.9013
Epoch 119/1000
70665/70665 [==============================] - 4s 52us/step - loss: 0.6373 - acc: 0.9030 - val_loss: 0.4809 - val_acc: 0.9009
Epoch 120/1000
70665/70665 [==============================] - 4s 50us/step - loss: 0.6393 - acc: 0.9041 - val_loss: 0.4858 - val_acc: 0.9002
Epoch 121/1000
70665/70665 [==============================] - 3s 48us/step - loss: 0.6380 - acc: 0.9038 - val_loss: 0.4888 - val_acc: 0.8979
Epoch 122/1000
70665/70665 [==============================] - 4s 50us/step - loss: 0.6378 - acc: 0.9040 - val_loss: 0.4904 - val_acc: 0.8966
Epoch 123/1000
70665/70665 [==============================] - 3s 49us/step - loss: 0.6382 - acc: 0.9028 - val_loss: 0.4803 - val_acc: 0.9018
Epoch 124/1000
70665/70665 [

70665/70665 [==============================] - 3s 49us/step - loss: 0.6238 - acc: 0.9064 - val_loss: 0.4817 - val_acc: 0.8938
Epoch 176/1000
70665/70665 [==============================] - 4s 51us/step - loss: 0.6225 - acc: 0.9061 - val_loss: 0.4772 - val_acc: 0.8974
Epoch 177/1000
70665/70665 [==============================] - 3s 49us/step - loss: 0.6233 - acc: 0.9057 - val_loss: 0.4692 - val_acc: 0.9030
Epoch 178/1000
70665/70665 [==============================] - 4s 50us/step - loss: 0.6238 - acc: 0.9056 - val_loss: 0.4754 - val_acc: 0.9006
Epoch 179/1000
70665/70665 [==============================] - 3s 48us/step - loss: 0.6243 - acc: 0.9054 - val_loss: 0.4860 - val_acc: 0.8959
Epoch 180/1000
70665/70665 [==============================] - 3s 49us/step - loss: 0.6266 - acc: 0.9019 - val_loss: 0.4703 - val_acc: 0.9038
Epoch 181/1000
70665/70665 [==============================] - 4s 51us/step - loss: 0.6262 - acc: 0.9037 - val_loss: 0.4695 - val_acc: 0.9030
Epoch 182/1000
70665/70665 [

70665/70665 [==============================] - 4s 52us/step - loss: 0.6154 - acc: 0.9048 - val_loss: 0.4621 - val_acc: 0.9069
Epoch 234/1000
70665/70665 [==============================] - 4s 50us/step - loss: 0.6167 - acc: 0.9067 - val_loss: 0.4728 - val_acc: 0.9023
Epoch 235/1000
70665/70665 [==============================] - 3s 47us/step - loss: 0.6153 - acc: 0.9056 - val_loss: 0.4667 - val_acc: 0.9040
Epoch 236/1000
70665/70665 [==============================] - 3s 50us/step - loss: 0.6150 - acc: 0.9060 - val_loss: 0.4665 - val_acc: 0.9044
Epoch 237/1000
70665/70665 [==============================] - 3s 49us/step - loss: 0.6147 - acc: 0.9067 - val_loss: 0.4784 - val_acc: 0.8996
Epoch 238/1000
70665/70665 [==============================] - 4s 50us/step - loss: 0.6149 - acc: 0.9069 - val_loss: 0.4724 - val_acc: 0.9026
Epoch 239/1000
70665/70665 [==============================] - 4s 51us/step - loss: 0.6154 - acc: 0.9063 - val_loss: 0.4702 - val_acc: 0.9032
Epoch 240/1000
70665/70665 [

...

70665/70665 [==============================] - 3s 49us/step - loss: 0.5618 - acc: 0.9108 - val_loss: 0.4494 - val_acc: 0.9032
Epoch 756/1000
70665/70665 [==============================] - 3s 49us/step - loss: 0.5619 - acc: 0.9098 - val_loss: 0.4503 - val_acc: 0.9032
Epoch 757/1000
70665/70665 [==============================] - 3s 45us/step - loss: 0.5650 - acc: 0.9110 - val_loss: 0.4490 - val_acc: 0.9040
Epoch 758/1000
70665/70665 [==============================] - 4s 51us/step - loss: 0.5642 - acc: 0.9097 - val_loss: 0.4449 - val_acc: 0.9063
Epoch 759/1000
70665/70665 [==============================] - 3s 49us/step - loss: 0.5660 - acc: 0.9087 - val_loss: 0.4382 - val_acc: 0.9093
Epoch 760/1000
70665/70665 [==============================] - 4s 51us/step - loss: 0.5627 - acc: 0.9116 - val_loss: 0.4384 - val_acc: 0.9092
Epoch 761/1000
70665/70665 [==============================] - 4s 52us/step - loss: 0.5608 - acc: 0.9129 - val_loss: 0.4427 - val_acc: 0.9084
Epoch 762/1000
70665/70665 [

70665/70665 [==============================] - 3s 49us/step - loss: 0.5590 - acc: 0.9131 - val_loss: 0.4455 - val_acc: 0.9043
Epoch 814/1000
70665/70665 [==============================] - 4s 53us/step - loss: 0.5619 - acc: 0.9100 - val_loss: 0.4404 - val_acc: 0.9074
Epoch 815/1000
70665/70665 [==============================] - 4s 52us/step - loss: 0.5592 - acc: 0.9115 - val_loss: 0.4302 - val_acc: 0.9145
Epoch 816/1000
70665/70665 [==============================] - 4s 53us/step - loss: 0.5605 - acc: 0.9136 - val_loss: 0.4436 - val_acc: 0.9047
Epoch 817/1000
70665/70665 [==============================] - 3s 49us/step - loss: 0.5615 - acc: 0.9096 - val_loss: 0.4369 - val_acc: 0.9105
Epoch 818/1000
70665/70665 [==============================] - 4s 50us/step - loss: 0.5661 - acc: 0.9111 - val_loss: 0.4536 - val_acc: 0.9000
Epoch 819/1000
70665/70665 [==============================] - 4s 52us/step - loss: 0.5627 - acc: 0.9091 - val_loss: 0.4557 - val_acc: 0.8959
Epoch 820/1000
70665/70665 [

70665/70665 [==============================] - 3s 48us/step - loss: 0.5587 - acc: 0.9117 - val_loss: 0.4423 - val_acc: 0.9049
Epoch 872/1000
70665/70665 [==============================] - 4s 50us/step - loss: 0.5582 - acc: 0.9110 - val_loss: 0.4391 - val_acc: 0.9092
Epoch 873/1000
70665/70665 [==============================] - 4s 51us/step - loss: 0.5604 - acc: 0.9100 - val_loss: 0.4434 - val_acc: 0.9065
Epoch 874/1000
70665/70665 [==============================] - 3s 47us/step - loss: 0.5580 - acc: 0.9122 - val_loss: 0.4690 - val_acc: 0.8869
Epoch 875/1000
70665/70665 [==============================] - 4s 51us/step - loss: 0.5584 - acc: 0.9127 - val_loss: 0.4505 - val_acc: 0.9006
Epoch 876/1000
70665/70665 [==============================] - 4s 52us/step - loss: 0.5594 - acc: 0.9104 - val_loss: 0.4327 - val_acc: 0.9092
Epoch 877/1000
70665/70665 [==============================] - 3s 48us/step - loss: 0.5591 - acc: 0.9122 - val_loss: 0.4428 - val_acc: 0.9075
Epoch 878/1000
70665/70665 [

70665/70665 [==============================] - 3s 48us/step - loss: 0.5604 - acc: 0.9107 - val_loss: 0.4431 - val_acc: 0.9052
Epoch 930/1000
70665/70665 [==============================] - 3s 49us/step - loss: 0.5590 - acc: 0.9126 - val_loss: 0.4344 - val_acc: 0.9108
Epoch 931/1000
70665/70665 [==============================] - 4s 52us/step - loss: 0.5588 - acc: 0.9121 - val_loss: 0.4426 - val_acc: 0.9067
Epoch 932/1000
70665/70665 [==============================] - 4s 52us/step - loss: 0.5602 - acc: 0.9102 - val_loss: 0.4455 - val_acc: 0.9043
Epoch 933/1000
70665/70665 [==============================] - 3s 47us/step - loss: 0.5596 - acc: 0.9115 - val_loss: 0.4462 - val_acc: 0.9046
Epoch 934/1000
70665/70665 [==============================] - 4s 51us/step - loss: 0.5591 - acc: 0.9102 - val_loss: 0.4595 - val_acc: 0.8925
Epoch 935/1000
70665/70665 [==============================] - 4s 51us/step - loss: 0.5603 - acc: 0.9107 - val_loss: 0.4510 - val_acc: 0.8961
Epoch 936/1000
70665/70665 [

70665/70665 [==============================] - 4s 50us/step - loss: 0.5553 - acc: 0.9125 - val_loss: 0.4400 - val_acc: 0.9077
Epoch 988/1000
70665/70665 [==============================] - 4s 50us/step - loss: 0.5587 - acc: 0.9109 - val_loss: 0.4424 - val_acc: 0.9045
Epoch 989/1000
70665/70665 [==============================] - 3s 48us/step - loss: 0.5560 - acc: 0.9116 - val_loss: 0.4409 - val_acc: 0.9077
Epoch 990/1000
70665/70665 [==============================] - 3s 49us/step - loss: 0.5557 - acc: 0.9117 - val_loss: 0.4423 - val_acc: 0.9060
Epoch 991/1000
70665/70665 [==============================] - 3s 48us/step - loss: 0.5555 - acc: 0.9120 - val_loss: 0.4425 - val_acc: 0.9051
Epoch 992/1000
70665/70665 [==============================] - 4s 50us/step - loss: 0.5545 - acc: 0.9123 - val_loss: 0.4380 - val_acc: 0.9088
Epoch 993/1000
70665/70665 [==============================] - 3s 49us/step - loss: 0.5543 - acc: 0.9128 - val_loss: 0.4374 - val_acc: 0.9089
Epoch 994/1000
70665/70665 [

l1=0005
adagrad
4.0
66/74, 84/98

In [67]:
precision_recall_fscore_support(y_test_class.values, classification_model.predict_classes(X_test))

(array([0.97854964, 0.49206949]),
 array([0.91478443, 0.80456931]),
 array([0.94559327, 0.61066198]),
 array([31567,  3239]))

In [68]:
precision_recall_fscore_support(y_train_class.values, classification_model.predict_classes(X_train))

(array([0.98247277, 0.52136752]),
 array([0.92139861, 0.83893643]),
 array([0.95095609, 0.64308305]),
 array([64121,  6544]))

In [ ]:
 classification_model.predict_proba(X_test)

## Regressor

In [ ]:
foo = pd.DataFrame(data=train_loss.values, columns=["loss"])

In [ ]:
X_r = pd.DataFrame(data=train_data).loc[foo[foo["loss"] > 0].index.values]

In [ ]:
y_r = foo.loc[foo[foo["loss"] > 0].index.values]

In [ ]:
y_r

In [ ]:
X_train_r, X_test_r,y_train_r, y_test_r =  train_test_split(X_r, y_r, test_size=0.33, random_state=42)

In [ ]:
def get_regression_model():
    model=Sequential()

    model.add(Dense(1024, input_dim=X_train_r.shape[1], kernel_regularizer=regularizers.l1_l2(l1=0.00001, l2=0.0)))
    model.add(Dropout(rate=0.2))
    model.add(Activation("tanh"))
    model.add(Dense(1024, kernel_regularizer=regularizers.l1_l2(l1=0.00001, l2=0.0)))
    model.add(Dropout(rate=0.2))
    model.add(Activation("tanh"))
    model.add(Dense(1))

    model.compile(optimizer='adagrad',
                  loss='mean_absolute_error')
    return model

regression_model = get_regression_model()

In [ ]:
regression_model.fit(X_train_r, y_train_r, epochs=200, batch_size=1024, validation_data=(X_test_r, y_test_r))

In [ ]:
regression_prediction = regression_model.predict(X_test_r).reshape(2160,)

In [ ]:
np.round(regression_prediction)[: 20]

In [ ]:
y_test_r.head(20)

In [ ]:
np.abs(regression_prediction - y_test_r["loss"].values).sum()/y_test_r.shape[0]

### Combined prediction

In [ ]:
class CombinedModel():
    def __init__(self,classification_model, regression_model):
        self.classification_model = classification_model
        self.regression_model = regression_model
        
    def predict(self,X):
        classification_prediction = self.classification_model.predict_classes(X)
        indices_of_predicted_defaults = np.where(classification_prediction == 1)[0]
        regression_prediction = self.regression_model.predict(X[indices_of_predicted_defaults])

        result = np.zeros(X.shape[0])
#         np.put(result, indices_of_predicted_defaults, regression_prediction.reshape(regression_prediction.shape[0], ))
        np.put(result, indices_of_predicted_defaults, np.full((regression_prediction.shape[0],), 2) )
        return result

In [ ]:
combined_model = CombinedModel(classification_model, regression_model)


In [ ]:
combined_prediction = combined_model.predict(train_data)

In [ ]:
combined_prediction

In [ ]:
np.abs(combined_prediction - train_loss.values).sum()/combined_prediction.shape[0]


### Submission to Kaggle

In [ ]:
#test_data = pd.read_csv("../data/loan-default-prediction/test_v2.csv")

In [ ]:
#test_ids = test_data["id"]

In [ ]:
#test_data.drop(columns=["id"], inplace=True)

In [ ]:
test_data = redundant_columns_remover.transform(test_data)
test_data = category_encoder.transform(test_data)
test_data = null_value_replacer.transform(test_data)
test_data = pca.transform(test_data)
test_data = standard_scaler.transform(test_data)

In [ ]:
predictions = combined_model.predict(test_data)

In [ ]:
predictions = predictions.reshape(34806,)

In [ ]:
predictions[:30]

In [ ]:
test_loss.values[:30]

In [ ]:
np.abs(predictions - test_loss.values).sum()/predictions.shape[0]


In [ ]:
pd.DataFrame(data={"loss":predictions}, columns=["loss"])

In [ ]:
to_submit = pd.concat([test_ids, pd.DataFrame(data={"loss":predictions}, columns=["loss"])], axis=1)

In [ ]:
to_submit.to_csv("../data/to_submit_3.csv", columns=["id", "loss"], index=False)

In [ ]:
to_submit

### Baselines for predictions

* [ ] do a baseline with mean absolute error against 0
* [ ] do a baseline with mae against existing loss distribution

In [ ]:
train_loss.mean() # This is also a loss against 0

In [ ]:
def calculate_mean_abs_error(predictions, actuals):
    return np.abs(predictions - actuals).sum()/predictions.shape[0]

In [ ]:
calculate_mean_abs_error(np.full(train_loss.shape, 0.8), train_loss) # This is loss against mean value

In [ ]:
random_dist = np.random.choice(
    train_loss.value_counts(normalize=True).sort_index().index.values,
    train_loss.shape[0],
    p=train_loss.value_counts(normalize=True).sort_index().values)

In [ ]:
calculate_mean_abs_error(random_dist, train_loss) # This is loss for random distribution with same prob values

In [ ]:
train_loss[train_loss> 0].mean()